In [4]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("AMomozZz/model")
config = AutoConfig.from_pretrained("AMomozZz/model")
print(config)
model = AutoModelForCausalLM.from_pretrained("AMomozZz/model", ignore_mismatched_sizes=True)

LlamaConfig {
  "_name_or_path": "AMomozZz/model",
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.46.3",
  "use_cache": true,
  "vocab_size": 32000
}



Some weights of LlamaForCausalLM were not initialized from the model checkpoint at AMomozZz/model and are newly initialized: ['lm_head.weight', 'model.layers.28.input_layernorm.weight', 'model.layers.28.mlp.down_proj.weight', 'model.layers.28.mlp.gate_proj.weight', 'model.layers.28.mlp.up_proj.weight', 'model.layers.28.post_attention_layernorm.weight', 'model.layers.28.self_attn.k_proj.weight', 'model.layers.28.self_attn.o_proj.weight', 'model.layers.28.self_attn.q_proj.weight', 'model.layers.28.self_attn.v_proj.weight', 'model.layers.29.input_layernorm.weight', 'model.layers.29.mlp.down_proj.weight', 'model.layers.29.mlp.gate_proj.weight', 'model.layers.29.mlp.up_proj.weight', 'model.layers.29.post_attention_layernorm.weight', 'model.layers.29.self_attn.k_proj.weight', 'model.layers.29.self_attn.o_proj.weight', 'model.layers.29.self_attn.q_proj.weight', 'model.layers.29.self_attn.v_proj.weight', 'model.layers.30.input_layernorm.weight', 'model.layers.30.mlp.down_proj.weight', 'model.l

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

In [7]:
# 输入文本
text = "Hello, how are you?"

# 编码输入
inputs = tokenizer(text, return_tensors="pt")

# 模型前向传播
outputs = model(**inputs)

# 获取最后一个隐藏层的输出
last_hidden_states = outputs.last_hidden_state
# print(last_hidden_states.shape)  # [batch_size, seq_length, hidden_size]
print(last_hidden_states)

IndexError: index out of range in self